In [2]:
#pip install seaborn

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler

# Data Loading

In [4]:
csv1 = pd.read_excel('data/Cushing Inventory.xlsx')  # Loading of cushing file 
csv2=pd.read_excel(open('data/Combined.xlsx', 'rb'),sheet_name='Main')  # Loading of pricing data


df_inventory=csv1[['SeriesDate', 'Inventory']].copy()

# Data processing of inventory data

In [5]:
df_inventory['SeriesDate'] = pd.to_datetime(df_inventory.SeriesDate, format='%Y/%M/%d')    
df_inventory=df_inventory.set_index('SeriesDate').resample('B').ffill().reset_index()
start_date = '2011-01-01'
mask = (df_inventory['SeriesDate'] > start_date)
df_inventory = df_inventory.loc[mask]
df_inventory.dropna(axis=0, inplace=True)
df_inventory.describe()

,Inventory
count,2425.000000
mean,44947.597113
std,13460.949383
min,17899.000000
25%,35501.000000
50%,44025.000000
75%,57047.000000
max,69420.000000


In [6]:
df_inventory.head()

,SeriesDate,Inventory
1756,2011-01-03,37493
1757,2011-01-04,37493
1758,2011-01-05,37493
1759,2011-01-06,37493
1760,2011-01-07,37376


# Data processing of pricing data

In [8]:
csv2.dropna(axis=0, inplace=True)   # dropping all NAN valued rows
csv2['Date'] = pd.to_datetime(csv2['Date'])      # converting Date to datetime format
csv2.describe()

,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,1&4,...,2&12,3&4,3&5,3&6,3&7,3&8,3&9,3&10,3&11,3&12
count,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000,...,2326.000000,2326.000000,2326.000000,2326.000000,2326.00000,2326.000000,2326.000000,2326.000000,2326.000000,2326.000000
mean,70.718865,68.744887,68.363298,77.972198,1.973978,2.355567,-7.253333,-0.345911,-0.573160,-0.718345,...,-0.001900,-0.145185,-0.227911,-0.244510,-0.21632,-0.158869,-0.079832,0.013960,0.115537,0.225348
std,23.470918,22.557033,22.491871,27.849996,4.629257,3.910296,6.781307,1.425895,1.943744,2.389033,...,4.431751,0.586318,1.120891,1.612702,2.05916,2.466086,2.841070,3.189049,3.515739,3.824173
min,-37.630000,10.490000,5.690000,11.740000,-55.520000,-43.320000,-49.370000,-58.060000,-63.910000,-66.140000,...,-18.060000,-2.920000,-4.740000,-6.040000,-7.17000,-8.170000,-8.980000,-9.700000,-10.360000,-10.940000
25%,50.585000,49.680000,49.745000,53.982500,-0.377500,0.160000,-9.535000,-0.540000,-1.080000,-1.560000,...,-2.910000,-0.480000,-0.920000,-1.290000,-1.57000,-1.800000,-1.960000,-2.130000,-2.267500,-2.380000
50%,63.745000,61.575000,59.960000,69.120000,1.400000,0.740000,-4.560000,-0.300000,-0.580000,-0.830000,...,-0.660000,-0.240000,-0.440000,-0.570000,-0.64000,-0.660000,-0.650000,-0.600000,-0.525000,-0.425000
75%,94.442500,91.517500,91.702500,106.305000,4.037500,3.610000,-2.342500,-0.020000,0.110000,0.340000,...,2.800000,0.230000,0.500000,0.807500,1.11000,1.427500,1.737500,2.020000,2.330000,2.617500
max,113.930000,119.530000,113.370000,130.780000,27.360000,21.610000,7.250000,2.660000,5.300000,6.080000,...,13.190000,1.770000,3.440000,5.080000,6.55000,7.740000,8.880000,9.770000,10.670000,11.450000


In [9]:
csv2.head()

,Date,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,...,2&12,3&4,3&5,3&6,3&7,3&8,3&9,3&10,3&11,3&12
0,2011-01-03,91.55,88.08,90.99,97.44,3.47,0.56,-5.89,-0.88,-1.59,...,-2.40,-0.58,-0.99,-1.31,-1.51,-1.63,-1.70,-1.75,-1.78,-1.69
1,2011-01-04,89.38,85.81,88.63,95.83,3.57,0.75,-6.45,-1.04,-1.94,...,-3.33,-0.75,-1.31,-1.74,-2.01,-2.19,-2.30,-2.39,-2.47,-2.43
2,2011-01-05,90.30,85.44,89.73,97.45,4.86,0.57,-7.15,-1.12,-2.09,...,-3.80,-0.84,-1.48,-1.99,-2.32,-2.53,-2.66,-2.76,-2.85,-2.83
3,2011-01-06,88.38,83.98,87.57,95.82,4.40,0.81,-7.44,-1.37,-2.52,...,-4.55,-0.94,-1.64,-2.19,-2.56,-2.82,-3.02,-3.19,-3.35,-3.40
4,2011-01-07,88.03,85.90,87.51,95.01,2.13,0.52,-6.98,-1.19,-2.14,...,-3.90,-0.81,-1.42,-1.90,-2.23,-2.47,-2.65,-2.79,-2.92,-2.95


# Merging both data frames and further processing

In [10]:
#combined=pd.merge(df_inventory,csv2, how='right', on='SeriesDate')
combined=pd.merge(
    df_inventory,
    csv2,
    left_on=['SeriesDate'],
    right_on=['Date']
)

In [11]:
%store combined

Stored 'combined' (DataFrame)


In [12]:
combined['year'] = pd.DatetimeIndex(combined['Date']).year
combined['month'] = pd.DatetimeIndex(combined['Date']).month
combined['day'] = pd.DatetimeIndex(combined['Date']).day
combined['weekday'] = pd.DatetimeIndex(combined['Date']).weekday   # Monday is 0 and Sunday is 6


In [13]:
# Drop features 
combined=combined.drop(columns=['SeriesDate','Date'])
combined.head()

,Inventory,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,...,3&7,3&8,3&9,3&10,3&11,3&12,year,month,day,weekday
0,37493,91.55,88.08,90.99,97.44,3.47,0.56,-5.89,-0.88,-1.59,...,-1.51,-1.63,-1.70,-1.75,-1.78,-1.69,2011,1,3,0
1,37493,89.38,85.81,88.63,95.83,3.57,0.75,-6.45,-1.04,-1.94,...,-2.01,-2.19,-2.30,-2.39,-2.47,-2.43,2011,1,4,1
2,37493,90.30,85.44,89.73,97.45,4.86,0.57,-7.15,-1.12,-2.09,...,-2.32,-2.53,-2.66,-2.76,-2.85,-2.83,2011,1,5,2
3,37493,88.38,83.98,87.57,95.82,4.40,0.81,-7.44,-1.37,-2.52,...,-2.56,-2.82,-3.02,-3.19,-3.35,-3.40,2011,1,6,3
4,37376,88.03,85.90,87.51,95.01,2.13,0.52,-6.98,-1.19,-2.14,...,-2.23,-2.47,-2.65,-2.79,-2.92,-2.95,2011,1,7,4


In [14]:
combined.describe()

,Inventory,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,...,3&7,3&8,3&9,3&10,3&11,3&12,year,month,day,weekday
count,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,...,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000,2322.000000
mean,44964.280792,70.839543,68.838297,68.459906,78.077499,2.001246,2.379637,-7.237956,-0.315728,-0.535306,...,-0.207084,-0.148140,-0.067804,0.027175,0.129836,0.240616,2015.146856,6.377261,15.678725,2.017227
std,13479.063880,23.291835,22.463492,22.389410,27.757631,4.476164,3.794915,6.730090,0.756542,1.402500,...,2.048463,2.454192,2.828266,3.175429,3.501406,3.809283,2.687591,3.446240,8.729659,1.396942
min,17899.000000,18.270000,14.130000,10.090000,13.060000,-9.520000,-2.760000,-29.610000,-7.290000,-11.760000,...,-5.500000,-6.180000,-6.740000,-7.320000,-7.900000,-8.510000,2011.000000,1.000000,1.000000,0.000000
25%,35501.000000,50.677500,49.770000,49.840000,54.062500,-0.367500,0.170000,-9.535000,-0.540000,-1.077500,...,-1.567500,-1.800000,-1.950000,-2.120000,-2.247500,-2.370000,2013.000000,3.000000,8.000000,1.000000
50%,44065.000000,63.810000,61.655000,59.980000,69.185000,1.410000,0.750000,-4.550000,-0.300000,-0.575000,...,-0.640000,-0.660000,-0.640000,-0.595000,-0.520000,-0.420000,2015.000000,6.000000,16.000000,2.000000
75%,57090.500000,94.450000,91.527500,91.730000,106.325000,4.047500,3.617500,-2.340000,-0.020000,0.110000,...,1.110000,1.437500,1.740000,2.027500,2.330000,2.620000,2017.000000,9.000000,23.000000,3.000000
max,69420.000000,113.930000,119.530000,113.370000,130.780000,27.360000,21.610000,7.250000,2.660000,5.300000,...,6.550000,7.740000,8.880000,9.770000,10.670000,11.450000,2020.000000,12.000000,31.000000,4.000000


# Getting Lag copies of all features

## Getting lag copies of input features

In [16]:
df_input_lag=combined.loc[:,'CL':'3&12']

In [13]:
df_input_lag.head()

,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,1&4,...,2&12,3&4,3&5,3&6,3&7,3&8,3&9,3&10,3&11,3&12
0,91.55,88.08,90.99,97.44,3.47,0.56,-5.89,-0.88,-1.59,-2.17,...,-2.40,-0.58,-0.99,-1.31,-1.51,-1.63,-1.70,-1.75,-1.78,-1.69
1,89.38,85.81,88.63,95.83,3.57,0.75,-6.45,-1.04,-1.94,-2.69,...,-3.33,-0.75,-1.31,-1.74,-2.01,-2.19,-2.30,-2.39,-2.47,-2.43
2,90.30,85.44,89.73,97.45,4.86,0.57,-7.15,-1.12,-2.09,-2.93,...,-3.80,-0.84,-1.48,-1.99,-2.32,-2.53,-2.66,-2.76,-2.85,-2.83
3,88.38,83.98,87.57,95.82,4.40,0.81,-7.44,-1.37,-2.52,-3.46,...,-4.55,-0.94,-1.64,-2.19,-2.56,-2.82,-3.02,-3.19,-3.35,-3.40
4,88.03,85.90,87.51,95.01,2.13,0.52,-6.98,-1.19,-2.14,-2.95,...,-3.90,-0.81,-1.42,-1.90,-2.23,-2.47,-2.65,-2.79,-2.92,-2.95


In [17]:
df_input_lag.count()

CL          2322
Bakken      2322
MID         2322
LLS         2322
Bakken.1    2322
MID.1       2322
LLS.1       2322
1&2         2322
1&3         2322
1&4         2322
1&5         2322
1&6         2322
1&7         2322
1&8         2322
1&9         2322
1&10        2322
1&11        2322
1&12        2322
2&3         2322
2&4         2322
2&5         2322
2&6         2322
2&7         2322
2&8         2322
2&9         2322
2&10        2322
2&11        2322
2&12        2322
3&4         2322
3&5         2322
3&6         2322
3&7         2322
3&8         2322
3&9         2322
3&10        2322
3&11        2322
3&12        2322
dtype: int64

In [18]:


lags = range(1, 50)  # 50 lags

df_input_lag=df_input_lag.assign(**{'{} (t-{})'.format(col, t): df_input_lag[col].shift(t)
    for t in lags
    for col in df_input_lag
})

In [16]:
df_input_lag.head()

,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,1&4,...,2&12 (t-49),3&4 (t-49),3&5 (t-49),3&6 (t-49),3&7 (t-49),3&8 (t-49),3&9 (t-49),3&10 (t-49),3&11 (t-49),3&12 (t-49)
0,91.55,88.08,90.99,97.44,3.47,0.56,-5.89,-0.88,-1.59,-2.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.38,85.81,88.63,95.83,3.57,0.75,-6.45,-1.04,-1.94,-2.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90.30,85.44,89.73,97.45,4.86,0.57,-7.15,-1.12,-2.09,-2.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88.38,83.98,87.57,95.82,4.40,0.81,-7.44,-1.37,-2.52,-3.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88.03,85.90,87.51,95.01,2.13,0.52,-6.98,-1.19,-2.14,-2.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_input_lag.count()

CL             2322
Bakken         2322
MID            2322
LLS            2322
Bakken.1       2322
               ... 
3&8 (t-49)     2273
3&9 (t-49)     2273
3&10 (t-49)    2273
3&11 (t-49)    2273
3&12 (t-49)    2273
Length: 1850, dtype: int64

In [17]:
df_input_lag.dropna(axis=0, inplace=True)   # dropping all NAN valued rows

In [20]:
df_input_lag.head()

,CL,Bakken,MID,LLS,Bakken.1,MID.1,LLS.1,1&2,1&3,1&4,...,2&12 (t-49),3&4 (t-49),3&5 (t-49),3&6 (t-49),3&7 (t-49),3&8 (t-49),3&9 (t-49),3&10 (t-49),3&11 (t-49),3&12 (t-49)
0,91.55,88.08,90.99,97.44,3.47,0.56,-5.89,-0.88,-1.59,-2.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,89.38,85.81,88.63,95.83,3.57,0.75,-6.45,-1.04,-1.94,-2.69,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,90.30,85.44,89.73,97.45,4.86,0.57,-7.15,-1.12,-2.09,-2.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,88.38,83.98,87.57,95.82,4.40,0.81,-7.44,-1.37,-2.52,-3.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,88.03,85.90,87.51,95.01,2.13,0.52,-6.98,-1.19,-2.14,-2.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_input_lag.count()

CL             2273
Bakken         2273
MID            2273
LLS            2273
Bakken.1       2273
               ... 
3&8 (t-49)     2273
3&9 (t-49)     2273
3&10 (t-49)    2273
3&11 (t-49)    2273
3&12 (t-49)    2273
Length: 1850, dtype: int64

In [20]:
df_input_lag.to_csv('input_data.csv')  #  saving data as csv file to be used with other modules

## Creating Lag copies of Output

In [21]:
out_columns=['Inventory']

In [22]:
df_inventory_lag=combined[out_columns]

In [23]:
lags = range(1, 50,6)  # 50 lags

df_inventory_lag=df_inventory_lag.assign(**{'{} (t-{})'.format(col, t): df_inventory_lag[col].shift(t)
    for t in lags
    for col in df_inventory_lag
})

In [24]:
df_inventory_lag.describe()

,Inventory,Inventory (t-1),Inventory (t-7),Inventory (t-13),Inventory (t-19),Inventory (t-25),Inventory (t-31),Inventory (t-37),Inventory (t-43),Inventory (t-49)
count,2322.000000,2321.000000,2315.000000,2309.000000,2303.000000,2297.000000,2291.000000,2285.000000,2279.000000,2273.000000
mean,44964.280792,44957.914261,44936.923110,44934.076223,44945.689101,44962.296038,44981.146224,44999.910722,45016.507240,45034.094589
std,13479.063880,13478.476105,13488.920215,13505.335520,13520.739009,13534.472284,13547.162838,13559.902825,13573.863689,13587.450249
min,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000,17899.000000
25%,35501.000000,35501.000000,35501.000000,35463.000000,35463.000000,35463.000000,35414.500000,35366.000000,35366.000000,35366.000000
50%,44065.000000,44065.000000,44025.000000,44025.000000,44065.000000,44065.000000,44087.000000,44087.000000,44124.000000,44127.000000
75%,57090.500000,57047.000000,57105.000000,57105.000000,57105.000000,57113.000000,57113.000000,57113.000000,57138.500000,57164.000000
max,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000,69420.000000


In [25]:
df_inventory_lag.count()

Inventory           2322
Inventory (t-1)     2321
Inventory (t-7)     2315
Inventory (t-13)    2309
Inventory (t-19)    2303
Inventory (t-25)    2297
Inventory (t-31)    2291
Inventory (t-37)    2285
Inventory (t-43)    2279
Inventory (t-49)    2273
dtype: int64

In [26]:
df_inventory_lag.dropna(axis=0, inplace=True)   # dropping all NAN valued rows

In [27]:
df_inventory_lag.head()

,Inventory,Inventory (t-1),Inventory (t-7),Inventory (t-13),Inventory (t-19),Inventory (t-25),Inventory (t-31),Inventory (t-37),Inventory (t-43),Inventory (t-49)
49,40020,40020.0,40263.0,37440.0,37657.0,37407.0,38334.0,36805.0,37376.0,37493.0
50,40020,40020.0,40263.0,38570.0,37657.0,37407.0,38334.0,37667.0,37376.0,37493.0
51,40020,40020.0,40263.0,38570.0,37657.0,37407.0,38334.0,37667.0,37376.0,37493.0
52,40197,40020.0,40263.0,38570.0,37440.0,37407.0,38334.0,37667.0,36805.0,37493.0
53,40197,40197.0,40263.0,38570.0,37440.0,37657.0,38334.0,37667.0,36805.0,37376.0


In [28]:
df_inventory_lag.to_csv('inventory_data.csv')  #  saving data as csv file to be used with other modules